<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#pnem-top3-환자들" data-toc-modified-id="pnem-top3-환자들-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>pnem top3 환자들</a></span></li><li><span><a href="#사망" data-toc-modified-id="사망-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>사망</a></span><ul class="toc-item"><li><span><a href="#lab-사망" data-toc-modified-id="lab-사망-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>lab 사망</a></span></li><li><span><a href="#pre-사망" data-toc-modified-id="pre-사망-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>pre 사망</a></span></li></ul></li><li><span><a href="#생존" data-toc-modified-id="생존-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>생존</a></span><ul class="toc-item"><li><span><a href="#lab-생존" data-toc-modified-id="lab-생존-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>lab 생존</a></span></li><li><span><a href="#pre-생존" data-toc-modified-id="pre-생존-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>pre 생존</a></span></li><li><span><a href="#pro-생존" data-toc-modified-id="pro-생존-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>pro 생존</a></span></li></ul></li></ul></div>

lab - VALUE <br>
pre - FORM_VAL_DISP <br>
pro -  시술을 진행함.<br>

In [1]:
mimic_path = '/data/MIMIC_III/'

import numpy as np
import pandas as pd

# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

#경고 에러 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
patient = pd.read_csv(f'{mimic_path}PATIENTS.csv')

In [3]:
lab = pd.read_csv(mimic_path + 'LABEVENTS.csv')
icd = pd.read_csv(mimic_path + 'D_ICD_DIAGNOSES.csv') # 질병 정보
patient_icd = pd.read_csv(mimic_path + 'DIAGNOSES_ICD.csv') # 환자별 질병 정보 

pre = pd.read_csv('/data/MIMIC_III/PRESCRIPTIONS.csv')
pro = pd.read_csv('/data/MIMIC_III/D_ITEMS.csv')

In [4]:
# 486
# 5070
# 48241
# 1. 폐렴 관련 질병 추출 -> 95가지 (pneu_list.nunique())
pneu_list = icd[(icd['SHORT_TITLE'].str.contains('pneum'))|icd['SHORT_TITLE'].str.contains('Pneum')]['ICD9_CODE']

# 2. 폐렴 관련 질병을 가진 환자 추출 -> 10559명 (all_pneu_patient.SUBJECT_ID.nunique())
all_pneu_patient = patient_icd[patient_icd['ICD9_CODE'].isin(pneu_list)]

# 3. top3_pneu에 해당하는 환자 추출 (top3_pneu는 환자가 가장 많이 갖고 있는 폐렴 관련 질병)
top3_pneu = all_pneu_patient['ICD9_CODE'].value_counts()[:3].index
top3_pneu_patient = all_pneu_patient[all_pneu_patient['ICD9_CODE'].isin(top3_pneu)]

# -----------------------------------------------------------------------------------------------------------------------

# 4. top3_pneu_patient에 한 환자 당 여러 병원 기록을 가지고 있을 수 있음 
# -> unique한 환자 수 : 7807명
환자list = top3_pneu_patient['SUBJECT_ID'].unique()

# 5. labevents에서 top3_pneu_patient의 정보 추출
환자lab = lab[lab['SUBJECT_ID'].isin(환자list)].reset_index(drop=True)

# 6. 'FLAG'열의 결측치를 문자열 'nan'으로 채운 후 분포 확인
# 'nan'으로 설정해주지 않으면 value_counts()에서 NaN의 개수가 안 잡힘
환자lab['FLAG'] = 환자lab['FLAG'].fillna('nan')
환자lab['FLAG'].value_counts()

# 7-0. 환자lab에서 abnormal 제거
# 환자lab = 환자lab[환자lab.FLAG != 'abnormal']

# 7. patient의 폐렴 환자수가 labevent의 폐렴 환자수보다 8명 더 많음
# 외래환자는 labevent의 'HADM_ID'를 가지지 않음.
# HADM_ID : Hospital Admission ID (입원ID)
환자lab_list = 환자lab['SUBJECT_ID'].unique()
외래환자idx = list(set(환자list) - set(환자lab['SUBJECT_ID'].unique()))
외래환자idx

nan         5931403
abnormal    3406609
delta         18337
Name: FLAG, dtype: int64

[60961, 48968, 17674, 9388, 19097, 93114, 58012, 95230]

In [5]:
# len(환자lab_list)
환자lab.FLAG.unique()

array(['nan', 'abnormal', 'delta'], dtype=object)

In [6]:
## float 확인
# for i in tmp.VALUE:
#     if type(i)==float:
#         continue
#     try:
#         _ = float(i)
#         continue
#     except:
#         print(i)

# pnem top3 환자들

In [7]:
top_sub = 환자lab_list

In [8]:
top_sub_df = patient[patient.SUBJECT_ID.isin(top_sub)]

dead = top_sub_df[top_sub_df.EXPIRE_FLAG==1].SUBJECT_ID.unique()
surv = top_sub_df[top_sub_df.EXPIRE_FLAG==0].SUBJECT_ID.unique()

# 사망

In [9]:
사망lab_item = [51277, 51006, 50912, 50893, 50882, 50983, 51003, 50862, 51274]
사망pre_item = [409176230]

## lab 사망

In [12]:
tmp_lab = lab[lab.SUBJECT_ID.isin(dead)]
tmp_lab = tmp_lab[tmp_lab.FLAG!='abnormal']
tmp_lab_사망 = tmp_lab[tmp_lab.ITEMID.isin(사망lab_item)]

li = []
for i in tmp_lab_사망.VALUE:
    try:
        li.append(float(i))
        continue
    except:
        try:
            li.append(float(i[1:]))
        except:
            li.append(np.NaN)
tmp_lab_사망.VALUE = li

# tmp_lab_사망
# tmp_lab_사망.groupby('ITEMID').mean()
tmp_lab_사망_pv = pd.pivot_table(tmp_lab_사망,
                                 columns = ['ITEMID'],
                                 values = ['VALUE'],
                                 aggfunc = 'describe')
tmp_lab_사망_pv

tmp_lab_사망.VALUE.nunique()

# for i in tmp_lab_사망_pv.columns:
#     pd.DataFrame(tmp_lab_사망_pv[i].value_counts()).sort_values(by = i, ascending = False)

ITEMID             50862          50882         50893         50912  \
VALUE 25%       3.600000      24.000000      8.600000      0.600000   
      50%       3.800000      26.000000      8.900000      0.800000   
      75%       4.100000      28.000000      9.200000      1.000000   
      count  8669.000000  116718.000000  73935.000000  91068.000000   
      max       5.200000      50.000000     10.300000      1.300000   
      mean      3.844607      26.106948      8.947769      0.802486   
      min       3.400000       5.000000      8.400000      0.000000   
      std       0.344031       2.768232      0.440739      0.231097   

ITEMID               50983        51003         51006         51274  \
VALUE 25%       137.000000     0.010000     11.000000     12.200000   
      50%       139.000000     0.010000     14.000000     12.700000   
      75%       142.000000     0.010000     17.000000     13.000000   
      count  145729.000000  5242.000000  57687.000000  24520.000000   
      max       148.000000    25.000000     20.000000    150.000000   
      mean      139.124752     0.014779     14.103334     12.594398   
      min       133.000000     0.000000      6.000000      9.400000   
      std         3.261251     0.345160      3.962329      1.859608   

ITEMID              51277  
VALUE 25%       13.800000  
      50%       14.500000  
      75%       15.000000  
      count  65267.000000  
      max       15.500000  
      mean      14.350021  
      min       11.300000  
      std        0.800422

147

In [22]:
tmp_lab_사망

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
3653,5153,9,150750.0,50983,2149-11-14 07:47:00,137.00,137.0,mEq/L,NaN
4323,4734,9,150750.0,50912,2149-11-09 11:20:00,1.20,1.2,mg/dL,NaN
4329,4740,9,150750.0,50983,2149-11-09 11:20:00,140.00,140.0,mEq/L,NaN
4331,4742,9,150750.0,51003,2149-11-09 11:20:00,0.01,NaN,ng/mL,NaN
4332,4743,9,150750.0,51006,2149-11-09 11:20:00,16.00,16.0,mg/dL,NaN
...,...,...,...,...,...,...,...,...,...
27852943,27772527,99063,166460.0,50882,2169-10-16 04:39:00,31.00,31.0,mEq/L,NaN
27852961,27772545,99063,166460.0,50912,2169-10-17 07:05:00,1.00,1.0,mg/dL,NaN
27852966,27772550,99063,166460.0,50983,2169-10-17 07:05:00,145.00,145.0,mEq/L,NaN
27852985,27772569,99063,166460.0,50912,2169-10-18 06:37:00,1.00,1.0,mg/dL,NaN


## pre 사망

In [15]:
tmp_pre = pre[pre.SUBJECT_ID.isin(dead)]
tmp_pre_사망 = tmp_pre[tmp_pre.NDC.isin(사망pre_item)]

tmp_pre_사망[tmp_pre_사망.NDC.isin(사망pre_item)]['PROD_STRENGTH'].value_counts()
tmp_pre_사망

2mg Syringe    4457
Name: PROD_STRENGTH, dtype: int64

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
4645,1425887,94,140037,213450.0,2176-09-03 00:00:00,2176-09-04 00:00:00,MAIN,Morphine Sulfate,Morphine Sulfate,Morphine Sulfate (Syringe),MORP2I,004070,409176230.0,2mg Syringe,1,mg,0.5,SYR,IV
4693,1425915,94,140037,213450.0,2176-09-09 00:00:00,2176-09-10 00:00:00,MAIN,Morphine Sulfate,Morphine Sulfate,Morphine Sulfate (Syringe),MORP2I,004070,409176230.0,2mg Syringe,1-2,mg,0.5-1,SYR,IV
4703,1425924,94,140037,213450.0,2176-09-10 00:00:00,2176-09-11 00:00:00,MAIN,Morphine Sulfate,Morphine Sulfate,Morphine Sulfate (Syringe),MORP2I,004070,409176230.0,2mg Syringe,1-2,mg,0.5-1,SYR,IV
4716,1436690,94,140037,213450.0,2176-09-11 00:00:00,2176-09-12 00:00:00,MAIN,Morphine Sulfate,Morphine Sulfate,Morphine Sulfate (Syringe),MORP2I,004070,409176230.0,2mg Syringe,2,mg,1,SYR,IV
4723,1436691,94,140037,213450.0,2176-09-12 00:00:00,2176-09-13 00:00:00,MAIN,Morphine Sulfate,Morphine Sulfate,Morphine Sulfate (Syringe),MORP2I,004070,409176230.0,2mg Syringe,2,mg,1,SYR,IV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4156153,149799,98853,189030,NaN,2187-02-03 00:00:00,2187-02-03 00:00:00,MAIN,Morphine Sulfate,Morphine Sulfate,Morphine Sulfate (Syringe),MORP2I,004070,409176230.0,2mg Syringe,0.1-0.5,mg,0.05-0.25,SYR,IV
4156157,149801,98853,189030,NaN,2187-02-03 00:00:00,2187-02-05 00:00:00,MAIN,Morphine Sulfate,Morphine Sulfate,Morphine Sulfate (Syringe),MORP2I,004070,409176230.0,2mg Syringe,0.25,mg,0.125,SYR,IV
4156161,149941,98853,189030,NaN,2187-02-05 00:00:00,2187-02-05 00:00:00,MAIN,Morphine Sulfate,Morphine Sulfate,Morphine Sulfate (Syringe),MORP2I,004070,409176230.0,2mg Syringe,0.5,mg,0.25,SYR,IV
4156163,149943,98853,189030,NaN,2187-02-05 00:00:00,2187-02-09 00:00:00,MAIN,Morphine Sulfate,Morphine Sulfate,Morphine Sulfate (Syringe),MORP2I,004070,409176230.0,2mg Syringe,0.25,mg,0.125,SYR,IV


# 생존

In [16]:
생존lab_item = [50861, 51249, 50878]
생존pre_item = [63323026201, 8084199, 409490234, 55390000401, 338070341, 574705050]
생존pro_item = [224275]

## lab 생존

In [17]:
tmp_lab = lab[lab.SUBJECT_ID.isin(surv)]
tmp_lab_생존 = tmp_lab[tmp_lab.ITEMID.isin(생존lab_item)]

li = []
for i in tmp_lab_생존.VALUE:
    try:
        li.append(float(i))
        continue
    except:
        try:
            li.append(float(i[1:]))
            
        except:
            if 'LESS THAN ' in i:
                try:
                    li.append(float(i.split('LESS THAN ')[-1]))
                except:
                    li.append(float(i.split('LESS THAN ')[-1].split(' ')[0]))
                
            elif 'GREATER THAN ' in i:
                try:
                    li.append(float(i.split('GREATER THAN ')[-1]))
                except:
                    li.append(float(i.split('GREATER THAN ')[-1].split(' ')[0]))
                
            else:
                li.append(np.NaN)

tmp_lab_생존.VALUE = li

# tmp_lab_생존
tmp_lab_생존.groupby('ITEMID').mean()

,ROW_ID,SUBJECT_ID,HADM_ID,VALUE,VALUENUM
ITEMID,,,,,
50861,1.508764e+07,35296.114370,148483.399089,147.887580,147.910034
50878,1.506440e+07,35246.170608,148497.798894,145.564293,145.564293
51249,1.510378e+07,35509.968867,149596.461603,33.411267,33.411267


## pre 생존

In [18]:
tmp_pre = pre[pre.SUBJECT_ID.isin(surv)]
tmp_pre_생존 = tmp_pre[tmp_pre.NDC.isin(생존pre_item)]
# tmp_pre_생존
tmp_pre_pv = pd.pivot(tmp_pre_생존,
                      columns = ['NDC'],
                      values = ['PROD_STRENGTH', 'DOSE_VAL_RX', 'FORM_VAL_DISP'])
# tmp_pre_pv
# tmp_pre_pv.columns
for i in tmp_pre_pv.columns:
    pd.DataFrame(tmp_pre_pv[i].value_counts()).sort_values(by = i, ascending = False)

,PROD_STRENGTH
,8084199.0
40mg Tablet,2068
40mg Tab,402
40MG TAB,156


,PROD_STRENGTH
,338070341.0
20mEq/50mL Premix,5986
20mEq/50ml Premix,493


,PROD_STRENGTH
,409490234.0
50mL Syringe,1994


,PROD_STRENGTH
,574705050.0
10mg Suppository,1977


,PROD_STRENGTH
,5.539000e+10
1mg Vial,1403


...

,FORM_VAL_DISP
,338070341.0
2,2692
1,2304
3,602
1-2,321
2-3,262
100,89
50,83
4,61
150,41


,FORM_VAL_DISP
,409490234.0
0.5,1699
1,273
2,9
5,5
0.5-1,4
0.25,1
4,1
1-2,1
0.29,1


,FORM_VAL_DISP
,574705050.0
1,1976
2,1


,FORM_VAL_DISP
,5.539000e+10
1,1377
5,12
10,6
25,4
20,2
2,2


,FORM_VAL_DISP
,6.332303e+10
1,3637
0.8,54
0.8-2.2,43
0.2,39
0.2-0.4,33
...,...
2.2,1
0.84-1.7,1
0.28-0.5,1


## pro 생존

In [19]:
pro[pro.ITEMID.isin(생존pro_item)]

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
9345,13081,224275,20 Gauge,20 Gauge,metavision,procedureevents_mv,Access Lines - Peripheral,None,Process,NaN


In [21]:
pro_mv = pd.read_csv('/data/MIMIC_III/PROCEDUREEVENTS_MV.csv')
tmp_pro = pro_mv[pro_mv.SUBJECT_ID.isin(surv)]
tmp_pro_생존 = tmp_pro[tmp_pro.ITEMID.isin(생존pro_item)]
pd.DataFrame(tmp_pro_생존.VALUE.value_counts()).sort_values('VALUE', ascending = False)

,VALUE
1.0,14
5760.0,10
4320.0,9
1350.0,7
350.0,6
...,...
1951.0,1
1026.0,1
814.0,1
1497.0,1
